In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('CLEAN_EDIT_model_data.csv')

In [3]:
df = df.drop(df.columns.to_series()["Unnamed: 0"], axis=1)

In [4]:
y = df.drop(df.columns.to_series()["retweet_from": "zoo"], axis=1)
y.head()

,signal_x
0,stay
1,stay
2,stay
3,stay
4,stay


In [5]:
y = pd.get_dummies(y).values

In [6]:
y

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1]], dtype=uint8)

In [7]:
x = df.drop(df.columns.to_series()["signal_x"], axis=1)
x.head()

,retweet_from,tweet_length,encoded_sentiment,polarity,tweet,ability,able,aboard,abort,aborted,...,you,your,yours,yourself,yr,yup,zero,zip,zone,zoo
0,137,105,1,0.366667,assuming acceleration of to but in a comfortab...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,64,113,1,0.200000,is capable of transporting satellite to orbit ...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,137,6,0,0.000000,yup,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,137,7,0,0.000000,part,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,137,96,1,0.650000,fly to most place on earth in under min and an...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
sentiment_data = x.drop(x.columns.to_series()["ability": "zoo"], axis=1)

In [9]:
tfidf_data = x.drop(x.columns.to_series()["retweet_from": "tweet"], axis=1)

In [10]:
tfidf_data.head()

,ability,able,aboard,abort,aborted,about,above,absolutely,absorb,absorbed,...,you,your,yours,yourself,yr,yup,zero,zip,zone,zoo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
from sklearn.decomposition import PCA

In [12]:
pca = PCA(n_components=50)

In [13]:
pca.fit(tfidf_data)

PCA(copy=True, iterated_power='auto', n_components=50, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [14]:
x_pca = pca.transform(tfidf_data)
x_pca

array([[-0.03053431, -0.02636362, -0.00967677, ...,  0.02809921,
         0.01081785, -0.01844455],
       [-0.0589778 , -0.11301196,  0.09692554, ...,  0.00336811,
        -0.02342973, -0.01792601],
       [-0.00220386,  0.04452286, -0.08819209, ..., -0.0975277 ,
        -0.01884974,  0.01580084],
       ...,
       [-0.01966206, -0.00474363, -0.04375533, ...,  0.05753096,
         0.01697662,  0.03687633],
       [-0.02563249, -0.00983638, -0.01822253, ..., -0.02616331,
        -0.02219404, -0.10793007],
       [-0.02191234, -0.00816946, -0.0320238 , ..., -0.02856638,
         0.02187234,  0.03395731]])

In [15]:
#tfidf ANN

In [16]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_pca, y, test_size = 0.2, random_state = 0)

In [18]:
classifier = Sequential()
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(units = 3, kernel_initializer='uniform', activation = 'softmax'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100, validation_split=0.1, validation_data=(X_test, y_test))

Train on 1761 samples, validate on 441 samples
Epoch 1/100
1761/1761 [==============================] - 1s 329us/step - loss: 0.6343 - acc: 0.6667 - val_loss: 0.6335 - val_acc: 0.6667
Epoch 2/100
1761/1761 [==============================] - 0s 120us/step - loss: 0.6309 - acc: 0.6667 - val_loss: 0.6330 - val_acc: 0.6667
Epoch 3/100
1761/1761 [==============================] - 0s 117us/step - loss: 0.6294 - acc: 0.6667 - val_loss: 0.6327 - val_acc: 0.6667
Epoch 4/100
1761/1761 [==============================] - 0s 108us/step - loss: 0.6284 - acc: 0.6669 - val_loss: 0.6324 - val_acc: 0.6674
Epoch 5/100
1761/1761 [==============================] - 0s 108us/step - loss: 0.6263 - acc: 0.6678 - val_loss: 0.6319 - val_acc: 0.6674
Epoch 6/100
1761/1761 [==============================] - 0s 105us/step - loss: 0.6248 - acc: 0.6703 - val_loss: 0.6318 - val_acc: 0.6644
Epoch 7/100
1761/1761 [==============================] - 0s 109us/step - loss: 0.6234 - acc: 0.6725 - val_loss: 0.6313 - val_acc: 0

1761/1761 [==============================] - 0s 128us/step - loss: 0.5975 - acc: 0.6894 - val_loss: 0.6546 - val_acc: 0.6531
Epoch 61/100
1761/1761 [==============================] - 0s 125us/step - loss: 0.5969 - acc: 0.6913 - val_loss: 0.6528 - val_acc: 0.6576
Epoch 62/100
1761/1761 [==============================] - 0s 127us/step - loss: 0.5954 - acc: 0.6926 - val_loss: 0.6518 - val_acc: 0.6576
Epoch 63/100
1761/1761 [==============================] - 0s 133us/step - loss: 0.5958 - acc: 0.6913 - val_loss: 0.6539 - val_acc: 0.6523
Epoch 64/100
1761/1761 [==============================] - 0s 116us/step - loss: 0.5932 - acc: 0.6926 - val_loss: 0.6610 - val_acc: 0.6455
Epoch 65/100
1761/1761 [==============================] - 0s 114us/step - loss: 0.5931 - acc: 0.6937 - val_loss: 0.6589 - val_acc: 0.6531
Epoch 66/100
1761/1761 [==============================] - 0s 119us/step - loss: 0.5915 - acc: 0.6928 - val_loss: 0.6569 - val_acc: 0.6531
Epoch 67/100
1761/1761 [=======================

In [19]:
#sentiment ANN

In [20]:
x = tfidf_data.values
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
sentiment_data = sentiment_data.values

In [26]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentiment_data, y, test_size = 0.2, random_state = 0)

In [27]:
classifier = Sequential()
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(units = 3, kernel_initializer='uniform', activation = 'softmax'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100, validation_split=0.1, validation_data=(X_test, y_test))

Train on 1761 samples, validate on 441 samples
Epoch 1/100


ValueError: could not convert string to float: 'been done'

###### 